# Gráficos de Barras Comparativos: Generales vs Ballotage

Este notebook genera **gráficos de barras comparativos** que muestran simultáneamente Congruente e Incongruente para ambas elecciones.

## Objetivo:

Comparar en un mismo gráfico las cuatro condiciones:
- **Congruente Generales** (verde claro)
- **Congruente Ballotage** (verde oscuro)
- **Incongruente Generales** (violeta claro)
- **Incongruente Ballotage** (violeta oscuro)

## Variables analizadas:

- **CO (Cambio de Opinión)**: Congruente vs Incongruente en Generales y Ballotage
- **CT (Cambio de Tiempo)**: Congruente vs Incongruente en Generales y Ballotage

## Agrupación:

Las barras se agrupan por congruencia:
- Grupo 1: Congruentes (Generales y Ballotage)
- Grupo 2: Incongruentes (Generales y Ballotage)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook', font_scale=1.1)

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Rutas a los archivos
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Generales = os.path.join(Ruta_Base, 'Generales_con_Congruencia.xlsx')
Archivo_Ballotage = os.path.join(Ruta_Base, 'Ballotage_con_Congruencia.xlsx')

# Cargar datos
df_Generales = pd.read_excel(Archivo_Generales)
df_Ballotage = pd.read_excel(Archivo_Ballotage)

print(f'✓ Datos cargados:')
print(f'  - Generales: {len(df_Generales)} registros')
print(f'  - Ballotage: {len(df_Ballotage)} registros')

# Verificar variables
vars_necesarias = ['CO_Congruente', 'CO_Incongruente', 'CT_Congruente', 'CT_Incongruente']

for nombre, df in [('Generales', df_Generales), ('Ballotage', df_Ballotage)]:
    faltantes = [v for v in vars_necesarias if v not in df.columns]
    if faltantes:
        print(f'  ⚠️  {nombre}: Faltan variables {faltantes}')
    else:
        print(f'  ✓ {nombre}: Todas las variables presentes')

## 2. Función de Visualización Comparativa

In [ ]:
def Crear_Grafico_Comparativo(df_gen, df_bal, variable_nombre, guardar=True):
    """
    Crea gráfico de barras comparativo para ambas elecciones.
    
    Parámetros:
    -----------
    df_gen : DataFrame con datos de Generales
    df_bal : DataFrame con datos de Ballotage
    variable_nombre : 'CO' o 'CT'
    guardar : bool, si True guarda el gráfico
    """
    
    # Determinar variables
    if variable_nombre == 'CO':
        var_cong = 'CO_Congruente'
        var_incong = 'CO_Incongruente'
        titulo_completo = 'Cambio de Opinión'
    else:
        var_cong = 'CT_Congruente'
        var_incong = 'CT_Incongruente'
        titulo_completo = 'Cambio de Tiempo'
    
    # Calcular estadísticas para las 4 condiciones
    # Congruentes
    cong_gen = df_gen[var_cong].dropna()
    cong_bal = df_bal[var_cong].dropna()
    
    # Incongruentes
    incong_gen = df_gen[var_incong].dropna()
    incong_bal = df_bal[var_incong].dropna()
    
    # Medias y errores
    medias = [
        cong_gen.mean(),
        cong_bal.mean(),
        incong_gen.mean(),
        incong_bal.mean()
    ]
    
    errores = [
        cong_gen.sem(),
        cong_bal.sem(),
        incong_gen.sem(),
        incong_bal.sem()
    ]
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 9))
    
    # Definir colores
    # Verdes: claro para Generales, oscuro para Ballotage
    # Violetas: claro para Generales, oscuro para Ballotage
    colores = [
        '#8dd3c7',  # Congruente Generales - verde claro
        '#66c2a5',  # Congruente Ballotage - verde oscuro
        '#bebada',  # Incongruente Generales - violeta claro
        '#9370db'   # Incongruente Ballotage - violeta oscuro
    ]
    
    # Posiciones de las barras
    # Agrupadas por congruencia: [0, 0.8] para congruentes, [2, 2.8] para incongruentes
    x_pos = np.array([0, 0.8, 2, 2.8])
    
    # Crear barras
    barras = ax.bar(x_pos, medias, yerr=errores, 
                     color=colores, 
                     alpha=0.85,
                     edgecolor='black',
                     linewidth=2,
                     width=0.7,
                     capsize=8,
                     error_kw={'linewidth': 2, 'elinewidth': 2, 'capthick': 2})
    
    # Configurar fondo con grid sutil
    ax.set_facecolor('#fafafa')
    ax.grid(True, alpha=0.25, linestyle='-', linewidth=0.5, color='gray', axis='y')
    ax.set_axisbelow(True)
    
    # Configurar ejes
    ax.set_xticks([0.4, 2.4])  # Centro de cada grupo
    ax.set_xticklabels(['Congruente', 'Incongruente'], 
                        fontsize=14, fontweight='bold')
    ax.set_ylabel('Media', fontsize=14, fontweight='bold')
    ax.tick_params(axis='y', labelsize=12)
    
    # Título
    ax.set_title(f'{titulo_completo}: Comparación Generales vs Ballotage\n' + 
                 'Congruente e Incongruente',
                 fontsize=16, fontweight='bold', pad=20)
    
    # Añadir línea en cero
    ax.axhline(y=0, color='red', linestyle='--', linewidth=1.5, alpha=0.6, zorder=0)
    
    # Añadir valores sobre las barras
    for i, (media, error) in enumerate(zip(medias, errores)):
        altura = media + error + (0.02 * (max(medias) - min(medias)))
        ax.text(x_pos[i], altura, f'{media:.3f}', 
                ha='center', va='bottom', 
                fontsize=10, fontweight='bold')
    
    # Tests estadísticos y significancia
    # Test 1: Congruente Gen vs Bal
    datos_pareados_cong = pd.DataFrame({
        'Gen': df_gen[var_cong],
        'Bal': df_bal[var_cong]
    }).dropna()
    
    if len(datos_pareados_cong) > 0:
        stat_cong, p_cong = stats.wilcoxon(datos_pareados_cong['Gen'], 
                                           datos_pareados_cong['Bal'])
        sig_cong = '***' if p_cong < 0.001 else '**' if p_cong < 0.01 else '*' if p_cong < 0.05 else 'ns'
    else:
        sig_cong = 'ns'
        p_cong = 1.0
    
    # Test 2: Incongruente Gen vs Bal
    datos_pareados_incong = pd.DataFrame({
        'Gen': df_gen[var_incong],
        'Bal': df_bal[var_incong]
    }).dropna()
    
    if len(datos_pareados_incong) > 0:
        stat_incong, p_incong = stats.wilcoxon(datos_pareados_incong['Gen'], 
                                               datos_pareados_incong['Bal'])
        sig_incong = '***' if p_incong < 0.001 else '**' if p_incong < 0.01 else '*' if p_incong < 0.05 else 'ns'
    else:
        sig_incong = 'ns'
        p_incong = 1.0
    
    # Añadir significancia para Congruentes
    if sig_cong != 'ns':
        y_max_cong = max(medias[0] + errores[0], medias[1] + errores[1])
        y_sig_cong = y_max_cong + 0.05 * (max(medias) - min(medias))
        
        ax.plot([0, 0.8], [y_sig_cong, y_sig_cong], 'k-', linewidth=1.5)
        ax.plot([0, 0], [y_sig_cong - 0.01, y_sig_cong], 'k-', linewidth=1.5)
        ax.plot([0.8, 0.8], [y_sig_cong - 0.01, y_sig_cong], 'k-', linewidth=1.5)
        ax.text(0.4, y_sig_cong + 0.01, sig_cong, ha='center', va='bottom', 
                fontsize=12, fontweight='bold')
    
    # Añadir significancia para Incongruentes
    if sig_incong != 'ns':
        y_max_incong = max(medias[2] + errores[2], medias[3] + errores[3])
        y_sig_incong = y_max_incong + 0.05 * (max(medias) - min(medias))
        
        ax.plot([2, 2.8], [y_sig_incong, y_sig_incong], 'k-', linewidth=1.5)
        ax.plot([2, 2], [y_sig_incong - 0.01, y_sig_incong], 'k-', linewidth=1.5)
        ax.plot([2.8, 2.8], [y_sig_incong - 0.01, y_sig_incong], 'k-', linewidth=1.5)
        ax.text(2.4, y_sig_incong + 0.01, sig_incong, ha='center', va='bottom', 
                fontsize=12, fontweight='bold')
    
    # Añadir texto con p-valores
    texto_pvalores = f'Congruente: p = {p_cong:.4f}\nIncongruente: p = {p_incong:.4f}'
    ax.text(0.02, 0.98, texto_pvalores, 
            transform=ax.transAxes,
            fontsize=10, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='gray'))
    
    # Crear leyenda fuera del gráfico
    from matplotlib.patches import Patch
    from matplotlib.lines import Line2D
    
    legend_elements = [
        Patch(facecolor='#8dd3c7', edgecolor='black', linewidth=2, 
              label='Congruente - Generales', alpha=0.85),
        Patch(facecolor='#66c2a5', edgecolor='black', linewidth=2, 
              label='Congruente - Ballotage', alpha=0.85),
        Patch(facecolor='#bebada', edgecolor='black', linewidth=2, 
              label='Incongruente - Generales', alpha=0.85),
        Patch(facecolor='#9370db', edgecolor='black', linewidth=2, 
              label='Incongruente - Ballotage', alpha=0.85),
        Line2D([0], [0], color='black', linewidth=2, marker='_', 
               markersize=15, label='Media ± Error Estándar'),
        Line2D([0], [0], color='red', linestyle='--', linewidth=1.5, 
               label='Línea de cero', alpha=0.6)
    ]
    
    # Posicionar leyenda fuera del área de ploteo
    ax.legend(handles=legend_elements, 
              loc='upper left', 
              bbox_to_anchor=(1.02, 1),
              fontsize=11,
              frameon=True,
              fancybox=True,
              shadow=True,
              framealpha=0.95)
    
    # Ajustar layout para que la leyenda no se corte
    plt.tight_layout()
    
    # Guardar como SVG
    if guardar:
        carpeta = 'Graficos_Comparativos'
        if not os.path.exists(carpeta):
            os.makedirs(carpeta)
        
        nombre_archivo = f'Comparativo_{variable_nombre}_Generales_vs_Ballotage.svg'
        ruta = os.path.join(carpeta, nombre_archivo)
        plt.savefig(ruta, format='svg', bbox_inches='tight', facecolor='white')
        print(f'✓ Gráfico guardado: {ruta}')
    
    plt.show()
    
    # Retornar resultados
    resultados = {
        'cong_gen_media': medias[0],
        'cong_bal_media': medias[1],
        'incong_gen_media': medias[2],
        'incong_bal_media': medias[3],
        'cong_gen_ee': errores[0],
        'cong_bal_ee': errores[1],
        'incong_gen_ee': errores[2],
        'incong_bal_ee': errores[3],
        'p_cong': p_cong,
        'sig_cong': sig_cong,
        'p_incong': p_incong,
        'sig_incong': sig_incong
    }
    
    return fig, ax, resultados

## 3. Estadísticas Descriptivas

In [ ]:
print('='*70)
print('ESTADÍSTICAS DESCRIPTIVAS - COMPARACIÓN GENERALES vs BALLOTAGE')
print('='*70)

for var_nombre, var_cong, var_incong in [
    ('CO', 'CO_Congruente', 'CO_Incongruente'),
    ('CT', 'CT_Congruente', 'CT_Incongruente')
]:
    print(f'\n{"="*70}')
    print(f'{var_nombre}:')
    print('='*70)
    
    # Congruentes
    cong_gen = df_Generales[var_cong].dropna()
    cong_bal = df_Ballotage[var_cong].dropna()
    
    print(f'\n  CONGRUENTE:')
    print(f'    Generales:')
    print(f'      n      = {len(cong_gen)}')
    print(f'      Media  = {cong_gen.mean():.4f}')
    print(f'      DE     = {cong_gen.std():.4f}')
    print(f'      EE     = {cong_gen.sem():.4f}')
    
    print(f'\n    Ballotage:')
    print(f'      n      = {len(cong_bal)}')
    print(f'      Media  = {cong_bal.mean():.4f}')
    print(f'      DE     = {cong_bal.std():.4f}')
    print(f'      EE     = {cong_bal.sem():.4f}')
    
    # Incongruentes
    incong_gen = df_Generales[var_incong].dropna()
    incong_bal = df_Ballotage[var_incong].dropna()
    
    print(f'\n  INCONGRUENTE:')
    print(f'    Generales:')
    print(f'      n      = {len(incong_gen)}')
    print(f'      Media  = {incong_gen.mean():.4f}')
    print(f'      DE     = {incong_gen.std():.4f}')
    print(f'      EE     = {incong_gen.sem():.4f}')
    
    print(f'\n    Ballotage:')
    print(f'      n      = {len(incong_bal)}')
    print(f'      Media  = {incong_bal.mean():.4f}')
    print(f'      DE     = {incong_bal.std():.4f}')
    print(f'      EE     = {incong_bal.sem():.4f}')

print('\n' + '='*70)

## 4. Gráficos Comparativos

### 4.1. Cambio de Opinión (CO)

In [ ]:
print('Generando gráfico comparativo para Cambio de Opinión...\n')

fig_co, ax_co, stats_co = Crear_Grafico_Comparativo(
    df_Generales,
    df_Ballotage,
    'CO'
)

### 4.2. Cambio de Tiempo (CT)

In [ ]:
print('Generando gráfico comparativo para Cambio de Tiempo...\n')

fig_ct, ax_ct, stats_ct = Crear_Grafico_Comparativo(
    df_Generales,
    df_Ballotage,
    'CT'
)

## 5. Tabla Resumen de Resultados

In [ ]:
# Crear tabla resumen
resumen = pd.DataFrame([
    {
        'Variable': 'CO',
        'Cong_Gen_Media': stats_co['cong_gen_media'],
        'Cong_Bal_Media': stats_co['cong_bal_media'],
        'Incong_Gen_Media': stats_co['incong_gen_media'],
        'Incong_Bal_Media': stats_co['incong_bal_media'],
        'p_Congruente': stats_co['p_cong'],
        'Sig_Congruente': stats_co['sig_cong'],
        'p_Incongruente': stats_co['p_incong'],
        'Sig_Incongruente': stats_co['sig_incong']
    },
    {
        'Variable': 'CT',
        'Cong_Gen_Media': stats_ct['cong_gen_media'],
        'Cong_Bal_Media': stats_ct['cong_bal_media'],
        'Incong_Gen_Media': stats_ct['incong_gen_media'],
        'Incong_Bal_Media': stats_ct['incong_bal_media'],
        'p_Congruente': stats_ct['p_cong'],
        'Sig_Congruente': stats_ct['sig_cong'],
        'p_Incongruente': stats_ct['p_incong'],
        'Sig_Incongruente': stats_ct['sig_incong']
    }
])

print('='*100)
print('RESUMEN DE RESULTADOS COMPARATIVOS')
print('='*100)
print('\n', resumen.to_string(index=False))
print('\n' + '='*100)

## 6. Guardar Resultados

In [ ]:
# Guardar tabla de resultados
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Comparativos')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

archivo = os.path.join(Carpeta_Salida, 'Resultados_Comparativos_Generales_vs_Ballotage.xlsx')
resumen.to_excel(archivo, index=False)
print(f'✓ Resultados guardados: {archivo}')

## 7. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: GRÁFICOS COMPARATIVOS GENERALES vs BALLOTAGE')
print('='*70)

print('\n📊 Análisis completado:')
print('  - Gráficos generados: 2 (CO y CT)')
print('  - Condiciones comparadas: 4 por gráfico')
print('    * Congruente Generales (verde claro)')
print('    * Congruente Ballotage (verde oscuro)')
print('    * Incongruente Generales (violeta claro)')
print('    * Incongruente Ballotage (violeta oscuro)')
print('  - Test estadístico: Wilcoxon pareado (Gen vs Bal)')

print('\n📁 Archivos generados:')
print('  Gráficos:')
print('    - Graficos_Comparativos/Comparativo_CO_Generales_vs_Ballotage.svg')
print('    - Graficos_Comparativos/Comparativo_CT_Generales_vs_Ballotage.svg')
print('  Datos:')
print('    - Data/Resultados_Comparativos/Resultados_Comparativos_Generales_vs_Ballotage.xlsx')

print('\n💡 Interpretación:')
print('  - Cada gráfico muestra 4 barras agrupadas por congruencia')
print('  - Verde claro/oscuro: Congruentes (Generales/Ballotage)')
print('  - Violeta claro/oscuro: Incongruentes (Generales/Ballotage)')
print('  - Las líneas de significancia comparan Generales vs Ballotage')
print('    dentro de cada tipo (Congruente o Incongruente)')

print('\n🎯 Hallazgos principales:')

for _, row in resumen.iterrows():
    print(f"\n  {row['Variable']}:")
    
    # Congruente
    if row['Sig_Congruente'] != 'ns':
        dir_cong = 'Generales > Ballotage' if row['Cong_Gen_Media'] > row['Cong_Bal_Media'] else 'Ballotage > Generales'
        print(f"    Congruente: {dir_cong} (p = {row['p_Congruente']:.4f} {row['Sig_Congruente']})")
    else:
        print(f"    Congruente: No hay diferencia significativa (p = {row['p_Congruente']:.4f})")
    
    # Incongruente
    if row['Sig_Incongruente'] != 'ns':
        dir_incong = 'Generales > Ballotage' if row['Incong_Gen_Media'] > row['Incong_Bal_Media'] else 'Ballotage > Generales'
        print(f"    Incongruente: {dir_incong} (p = {row['p_Incongruente']:.4f} {row['Sig_Incongruente']})")
    else:
        print(f"    Incongruente: No hay diferencia significativa (p = {row['p_Incongruente']:.4f})")

print('\n' + '='*70)
print('✓ ANÁLISIS COMPLETADO')
print('='*70)